In [1]:
import yaml
import os
import numpy as np

In [2]:
input_folder = "./k8s_raw/"
output_folder= "./k8s_converted/"

In [3]:
qualities = [1,2,3]
instances = [1,2,3,4,5]
edge_servers=[
    (1, "edge-1"),
    (2, "edge-2"),
    (3, "edge-3"),
    (4, "edge-4"),
    (5, "edge-5"),
    (6, "edge-6"),
    (7, "edge-7"),
    (8, "edge-8"),
    (9, "edge-9"),
    (10, "edge-10")
]


In [7]:
try:
    os.mkdir(output_folder)
except:
    os.system("rm -rf "+output_folder)
    os.mkdir(output_folder)

In [5]:
config_files = os.listdir(input_folder)

In [6]:
for file in config_files:
    with open(input_folder+file, "r") as f:
        config = list(yaml.safe_load_all(f))
        service_name = file.replace(".yaml","")

        for q in qualities:
            for i in instances:
                for s in edge_servers:
                    svc_label = "-%d-%d-%d" % (q, i, s[0])
                    # Update service name
                    config[0]["metadata"]["name"]+=svc_label
                    config[0]["metadata"]["labels"]["name"] +=svc_label

                    config[1]["metadata"]["name"]+=svc_label
                    config[1]["spec"]["selector"]["matchLabels"]["app"]+=svc_label
                    # set init replica to 0
                    config[1]["spec"]["replicas"] = 0
                    config[1]["spec"]["template"]["metadata"]["labels"]["app"] +=svc_label
                    # set node to edge node name
                    config[1]["spec"]["template"]["spec"]["nodeName"] = s[1]
                    config[1]["spec"]["template"]["spec"]["containers"][0]["env"][0]["value"]+=svc_label
                    # Add cpu assignment
                    config[1]["spec"]["template"]["spec"]["containers"][0]["env"].append({'name':"CPU_REQUEST",'value':int(int(config[1]["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]["cpu"].replace("m",""))/100)})
                    # Set initial delay
                    config[1]["spec"]["template"]["spec"]["containers"][0]["readinessProbe"]["initialDelaySeconds"] = "0"
                    config[1]["spec"]["template"]["spec"]["volumes"][0]["configMap"]["name"]+=svc_label

                    config[2]["metadata"]["name"]+=svc_label
                    config[2]["spec"]["selector"]["app"]+=svc_label

                    # output
                    output_file_name=service_name+svc_label+".yaml"
                    yaml.safe_dump_all(config)
                    with open(output_folder+output_file_name,"w") as f2:
                        yaml.safe_dump_all(config,f2,)





